In [1]:
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
import pandas as pd
import numpy as np
import sys

Lectura de la base de CURT correspondiente al municipio (Tipo shp)

In [2]:
CURT_mun=gpd.read_file(r"D:\Secretaría\Bases_shp_INEGI\CURT\024 Atlacomulco\ATLACOMULCO_PREDIOS.shp")
CURT_mun

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-99.87493 19.79576, -99.87494 19.795..."
1,15,024,0000,0240109128000000,194737177109953100419,NaN,"POLYGON ((-99.88603 19.79363, -99.88612 19.793..."
2,15,024,0000,0240109116000000,194736032409953102406,NaN,"POLYGON ((-99.88603 19.79363, -99.88590 19.793..."
3,15,024,0000,0240142005000000,194824758809952102202,NaN,"POLYGON ((-99.86944 19.80691, -99.86948 19.806..."
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-99.87493 19.79576, -99.87485 19.795..."
...,...,...,...,...,...,...,...
1919,15,024,0000,0240122320000000,194719536809952058052,NaN,"POLYGON ((-99.86814 19.78877, -99.86809 19.788..."
1920,15,024,0000,0240122321000000,194720252909952063672,NaN,"POLYGON ((-99.86820 19.78889, -99.86814 19.788..."
1921,15,024,0000,0240122328000000,194720252909952058206,NaN,"POLYGON ((-99.86827 19.78906, -99.86820 19.788..."
1922,15,024,0000,0240122345000000,194720239609952072994,NaN,"POLYGON ((-99.86859 19.78904, -99.86872 19.788..."


In [3]:
CURT_mun['CLAVECATASTRAL']= CURT_mun['id_cat']
CURT_mun['CLAVE_CATASTRAL']=CURT_mun['CLAVECATASTRAL'].str.slice(0,10)
CURT_mun


,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,CLAVECATASTRAL,CLAVE_CATASTRAL
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-99.87493 19.79576, -99.87494 19.795...",0240144801000000,0240144801
1,15,024,0000,0240109128000000,194737177109953100419,NaN,"POLYGON ((-99.88603 19.79363, -99.88612 19.793...",0240109128000000,0240109128
2,15,024,0000,0240109116000000,194736032409953102406,NaN,"POLYGON ((-99.88603 19.79363, -99.88590 19.793...",0240109116000000,0240109116
3,15,024,0000,0240142005000000,194824758809952102202,NaN,"POLYGON ((-99.86944 19.80691, -99.86948 19.806...",0240142005000000,0240142005
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-99.87493 19.79576, -99.87485 19.795...",0240144837000000,0240144837
...,...,...,...,...,...,...,...,...,...
1919,15,024,0000,0240122320000000,194719536809952058052,NaN,"POLYGON ((-99.86814 19.78877, -99.86809 19.788...",0240122320000000,0240122320
1920,15,024,0000,0240122321000000,194720252909952063672,NaN,"POLYGON ((-99.86820 19.78889, -99.86814 19.788...",0240122321000000,0240122321
1921,15,024,0000,0240122328000000,194720252909952058206,NaN,"POLYGON ((-99.86827 19.78906, -99.86820 19.788...",0240122328000000,0240122328
1922,15,024,0000,0240122345000000,194720239609952072994,NaN,"POLYGON ((-99.86859 19.78904, -99.86872 19.788...",0240122345000000,0240122345


In [4]:
CURT_mun = CURT_mun.set_geometry('geometry')
CURT_mun['geometry']=CURT_mun.centroid

In [5]:
#CURT_I['CLAVE_CATASTRAL']=CURT_I['CLAVE_CATASTRAL'].str.slice(0,10)
CURT_mun['LATITUD']= CURT_mun['geometry'].y
CURT_mun['LONGITUD']= CURT_mun['geometry'].x
CURT_mun

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,CLAVECATASTRAL,CLAVE_CATASTRAL,LATITUD,LONGITUD
0,15,024,0000,0240144801000000,194744919109952299131,NaN,POINT (-99.87498 19.79582),0240144801000000,0240144801,19.795816,-99.874977
1,15,024,0000,0240109128000000,194737177109953100419,NaN,POINT (-99.88612 19.79366),0240109128000000,0240109128,19.793660,-99.886123
2,15,024,0000,0240109116000000,194736032409953102406,NaN,POINT (-99.88618 19.79346),0240109116000000,0240109116,19.793464,-99.886179
3,15,024,0000,0240142005000000,194824758809952102202,NaN,POINT (-99.86951 19.80688),0240142005000000,0240142005,19.806878,-99.869506
4,15,024,0000,0240144837000000,194744232409952299340,NaN,POINT (-99.87497 19.79567),0240144837000000,0240144837,19.795668,-99.874970
...,...,...,...,...,...,...,...,...,...,...,...
1919,15,024,0000,0240122320000000,194719536809952058052,NaN,POINT (-99.86827 19.78876),0240122320000000,0240122320,19.788760,-99.868271
1920,15,024,0000,0240122321000000,194720252909952063672,NaN,POINT (-99.86837 19.78889),0240122321000000,0240122321,19.788893,-99.868374
1921,15,024,0000,0240122328000000,194720252909952058206,NaN,POINT (-99.86834 19.78903),0240122328000000,0240122328,19.789027,-99.868342
1922,15,024,0000,0240122345000000,194720239609952072994,NaN,POINT (-99.86869 19.78896),0240122345000000,0240122345,19.788957,-99.868694


In [19]:
CURT_mun.to_file(r'D:\Secretaría\cruces_bases\Atlacomulco/curt_atlacomulco_completo.shp')

Normalized/laundered field name: 'CLAVECATASTRAL' to 'CLAVECATAS'
Normalized/laundered field name: 'CLAVE_CATASTRAL' to 'CLAVE_CATA'


In [11]:
Predial_Ixta = pd.read_excel(r"D:\Secretaría\cruces_bases\ANEXO 3 PRED DICIEMBRE 2022 - Arturo Baez.xlsx")
Predial_Ixta

,No. CONSECUTIVO,CLAVECATASTRAL,DOMICILIO,NOMBRE DEL PROPIETARIO,RFC,ESTATUS DE LA CLAVE,VALOR CATASTRAL,SUPERFICIE DE TERRENO,SUPERFICIE CONSTRUCCION,USO DE ÁREA HOMOGÉNEA,PERIODO DE RECAUDACION CORRIENTE REZAGO,PERIODO DE RECAUDACION CORRIENTE,EJERCICIO FISCAL
0,1,0600012209500000,CAMINO REAL A TONATICO S/N EL SALITRE,GARCIA SERRANO SALVADOR,NaN,2,113.0,0,NaN,0.00,12,2016,2016.0
1,2,0600100102000000,PLAZA DE LOS MARTIRES COL CENTRO CP 51900 PALA...,MUNICIPIO DE IXTAPAN DE LA SAL,NaN,2,462.0,924,NaN,691.14,12,2016,2016.0
2,3,0600100103000000,INDEPENDENCIA OTE CENTRO,TENORIO DOMINGUEZ HERMELINDA,NaN,2,213.0,718,NaN,8259.92,12,2022,2022.0
3,4,0600100104000000,"CALLE INDEPENDENCIA NO.101COL. CENTRO,",FIGUEROA DIAZ GUSTAVO,FIDG002120,2,283.0,577,NaN,6371.40,12,2022,2022.0
4,5,0600100105000000,NaN,BENJAMIN IGNACIO ARAUJO DELGADO,ARDB630125,2,170.0,178,NaN,1297.60,12,2022,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21512,21513,0607101080000000,"20 DE NOVIEMBRE NO.24 BARRIO DE SANTACATARINA,...",HERNANDEZ SALDAÑA MARIA,NaN,2,196.0,0,NaN,0.00,12,2004,NaN
21513,21514,0607122130000000,"CALLE S/N COL. CEDROS ,IXTAPAN DE LA SAL",FLORES CASTAÑO SALOME,NaN,2,485.0,0,NaN,0.00,12,2004,NaN
21514,21515,0608012280000000,CALLE SIN NOMBRE SN YERBAS BUENAS,RUIZ GUADARRAMA EVODIO,NaN,2,1082.0,0,NaN,0.00,12,2016,NaN
21515,21516,0690030030301004,NaN,CLUB DE AIRE DE IXTAPAN DE LA SAL,NaN,2,284.0,0,NaN,0.00,12,2011,NaN


In [12]:
Predial_Ixta['CLAVE_CATASTRAL']=Predial_Ixta.CLAVECATASTRAL.str.slice(0,10)
Predial_Ixta

,No. CONSECUTIVO,CLAVECATASTRAL,DOMICILIO,NOMBRE DEL PROPIETARIO,RFC,ESTATUS DE LA CLAVE,VALOR CATASTRAL,SUPERFICIE DE TERRENO,SUPERFICIE CONSTRUCCION,USO DE ÁREA HOMOGÉNEA,PERIODO DE RECAUDACION CORRIENTE REZAGO,PERIODO DE RECAUDACION CORRIENTE,EJERCICIO FISCAL,CLAVE_CATASTRAL
0,1,0600012209500000,CAMINO REAL A TONATICO S/N EL SALITRE,GARCIA SERRANO SALVADOR,NaN,2,113.0,0,NaN,0.00,12,2016,2016.0,0600012209
1,2,0600100102000000,PLAZA DE LOS MARTIRES COL CENTRO CP 51900 PALA...,MUNICIPIO DE IXTAPAN DE LA SAL,NaN,2,462.0,924,NaN,691.14,12,2016,2016.0,0600100102
2,3,0600100103000000,INDEPENDENCIA OTE CENTRO,TENORIO DOMINGUEZ HERMELINDA,NaN,2,213.0,718,NaN,8259.92,12,2022,2022.0,0600100103
3,4,0600100104000000,"CALLE INDEPENDENCIA NO.101COL. CENTRO,",FIGUEROA DIAZ GUSTAVO,FIDG002120,2,283.0,577,NaN,6371.40,12,2022,2022.0,0600100104
4,5,0600100105000000,NaN,BENJAMIN IGNACIO ARAUJO DELGADO,ARDB630125,2,170.0,178,NaN,1297.60,12,2022,2022.0,0600100105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21512,21513,0607101080000000,"20 DE NOVIEMBRE NO.24 BARRIO DE SANTACATARINA,...",HERNANDEZ SALDAÑA MARIA,NaN,2,196.0,0,NaN,0.00,12,2004,NaN,0607101080
21513,21514,0607122130000000,"CALLE S/N COL. CEDROS ,IXTAPAN DE LA SAL",FLORES CASTAÑO SALOME,NaN,2,485.0,0,NaN,0.00,12,2004,NaN,0607122130
21514,21515,0608012280000000,CALLE SIN NOMBRE SN YERBAS BUENAS,RUIZ GUADARRAMA EVODIO,NaN,2,1082.0,0,NaN,0.00,12,2016,NaN,0608012280
21515,21516,0690030030301004,NaN,CLUB DE AIRE DE IXTAPAN DE LA SAL,NaN,2,284.0,0,NaN,0.00,12,2011,NaN,0690030030


In [13]:
Predial_Ixta_2022=Predial_Ixta[Predial_Ixta['EJERCICIO FISCAL'] == 2022]
Predial_Ixta_2022

,No. CONSECUTIVO,CLAVECATASTRAL,DOMICILIO,NOMBRE DEL PROPIETARIO,RFC,ESTATUS DE LA CLAVE,VALOR CATASTRAL,SUPERFICIE DE TERRENO,SUPERFICIE CONSTRUCCION,USO DE ÁREA HOMOGÉNEA,PERIODO DE RECAUDACION CORRIENTE REZAGO,PERIODO DE RECAUDACION CORRIENTE,EJERCICIO FISCAL,CLAVE_CATASTRAL
2,3,0600100103000000,INDEPENDENCIA OTE CENTRO,TENORIO DOMINGUEZ HERMELINDA,NaN,2,213.0,718,NaN,8259.92,12,2022,2022.0,0600100103
3,4,0600100104000000,"CALLE INDEPENDENCIA NO.101COL. CENTRO,",FIGUEROA DIAZ GUSTAVO,FIDG002120,2,283.0,577,NaN,6371.40,12,2022,2022.0,0600100104
4,5,0600100105000000,NaN,BENJAMIN IGNACIO ARAUJO DELGADO,ARDB630125,2,170.0,178,NaN,1297.60,12,2022,2022.0,0600100105
5,6,0600100106000000,J. MINA NO 5,ROSALINDA SOTELO DIAZ LEAL,NaN,2,96.0,177,NaN,2532.12,12,2022,2022.0,0600100106
6,7,0600100107000000,ALLENDE,DIAZ MONTERO RUTH,DIDS001631,2,55.0,100,NaN,597.64,12,2022,2022.0,0600100107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21433,21434,0600905819000000,NaN,TRUJILLO LOPEZ ALBERTO,TRLA,2,2500.0,0,NaN,175.38,12,2022,2022.0,0600905819
21438,21439,0600905824000000,NaN,LOPEZ GUADARRAMA ISSAC,LOGI,2,5662.0,0,NaN,181.81,12,2022,2022.0,0600905824
21439,21440,0600905825000000,NaN,BELTRAN DE BERNAL LILIA,BEDBL,2,2129.0,0,NaN,174.49,12,2021,2022.0,0600905825
21440,21441,0600905826000000,NaN,ROJAS COLIN MELCHOR,ROCM1,2,6591.0,0,NaN,0.00,12,2006,2022.0,0600905826


In [15]:
Predial_Ixta_2022.columns[Predial_Ixta_2022.columns.isin(CURT_mun.columns)]
falta= Predial_Ixta_2022.columns[~Predial_Ixta_2022.columns.isin(CURT_mun.columns)]
falta

Index(['No. CONSECUTIVO', 'DOMICILIO', 'NOMBRE DEL PROPIETARIO', 'RFC',
       'ESTATUS DE LA CLAVE', 'VALOR CATASTRAL', 'SUPERFICIE DE TERRENO',
       'SUPERFICIE CONSTRUCCION', 'USO DE ÁREA HOMOGÉNEA',
       'PERIODO DE RECAUDACION CORRIENTE REZAGO',
       'PERIODO DE RECAUDACION CORRIENTE', 'EJERCICIO FISCAL'],
      dtype='object')

In [16]:
cruce_cat=pd.merge(Predial_Ixta_2022[['CLAVE_CATASTRAL','No. CONSECUTIVO', 'DOMICILIO', 'NOMBRE DEL PROPIETARIO', 'RFC',
       'ESTATUS DE LA CLAVE', 'VALOR CATASTRAL', 'SUPERFICIE DE TERRENO',
       'USO DE ÁREA HOMOGÉNEA', 'PERIODO DE RECAUDACION CORRIENTE REZAGO',
       'PERIODO DE RECAUDACION CORRIENTE', 'EJERCICIO FISCAL']],CURT_mun, on="CLAVE_CATASTRAL", how='outer', indicator=True )

In [17]:
Ixta_curt = cruce_cat

In [18]:
Ixta_curt

,CLAVE_CATASTRAL,No. CONSECUTIVO,DOMICILIO,NOMBRE DEL PROPIETARIO,RFC,ESTATUS DE LA CLAVE,VALOR CATASTRAL,SUPERFICIE DE TERRENO,USO DE ÁREA HOMOGÉNEA,PERIODO DE RECAUDACION CORRIENTE REZAGO,...,cve_mun,cve_loc,id_cat,curt,notas,geometry,CLAVECATASTRAL,LATITUD,LONGITUD,_merge
0,0600100103,3.0,INDEPENDENCIA OTE CENTRO,TENORIO DOMINGUEZ HERMELINDA,NaN,2.0,213.0,718.0,8259.92,12.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only
1,0600100104,4.0,"CALLE INDEPENDENCIA NO.101COL. CENTRO,",FIGUEROA DIAZ GUSTAVO,FIDG002120,2.0,283.0,577.0,6371.40,12.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only
2,0600100105,5.0,NaN,BENJAMIN IGNACIO ARAUJO DELGADO,ARDB630125,2.0,170.0,178.0,1297.60,12.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only
3,0600100106,6.0,J. MINA NO 5,ROSALINDA SOTELO DIAZ LEAL,NaN,2.0,96.0,177.0,2532.12,12.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only
4,0600100107,7.0,ALLENDE,DIAZ MONTERO RUTH,DIDS001631,2.0,55.0,100.0,597.64,12.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13849,0240122320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,024,0000,0240122320000000,194719536809952058052,NaN,POINT (-99.86827 19.78876),0240122320000000,19.788760,-99.868271,right_only
13850,0240122321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,024,0000,0240122321000000,194720252909952063672,NaN,POINT (-99.86837 19.78889),0240122321000000,19.788893,-99.868374,right_only
13851,0240122328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,024,0000,0240122328000000,194720252909952058206,NaN,POINT (-99.86834 19.78903),0240122328000000,19.789027,-99.868342,right_only
13852,0240122345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,024,0000,0240122345000000,194720239609952072994,NaN,POINT (-99.86869 19.78896),0240122345000000,19.788957,-99.868694,right_only


In [19]:
Ixta_curt = Ixta_curt.drop('_merge', axis=1)


In [43]:
Ixta_curt[Ixta_curt.CLAVE_CATASTRAL.isna()]

,CLAVE_CATASTRAL,No. CONSECUTIVO,DOMICILIO,NOMBRE DEL PROPIETARIO,RFC,ESTATUS DE LA CLAVE,VALOR CATASTRAL,SUPERFICIE DE TERRENO,USO DE ÁREA HOMOGÉNEA,PERIODO DE RECAUDACION CORRIENTE REZAGO,...,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,CLAVECATASTRAL,LATITUD,LONGITUD


In [ ]:
cruce_cat.to_excel(r"C:\Users\mfpen\OneDrive\Documentos\Secretaría\cruces_predial_agua/cruce_curt_predial_Ixtapan1.xlsx")

In [20]:
pred_pred=pd.read_excel(r"D:\Secretaría\cruces_bases\Ixtapan de La Sal_PREDIAL.xlsx")
pred_pred

,CLAVECATASTRAL,CLAVEASENTAMIENTO,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CLAVEMANZANA,CLAVEZONA,CLAVESTATUS,DIRECCION,VALORTERRENO,...,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,LATITUD,LONGITUD,FECHAALTA,ASENTAMIENTO_NR,CLAVECP_NR
0,0600100101000000,1517,51900,40,15,1,1,1,PLAZA LOS MARTIRES COL CENTRO S/N,3121950,...,3147,0,133,0,,0,0,2020-10-07 14:45:58.0,,
1,0600100102000000,1517,51900,40,15,1,1,1,PLAZA DE LOS MARTIRES COL CENTRO CP 51900 PALA...,1086957,...,462,0,876,0,,0,0,2014-12-03 13:11:37.0,,
2,0600100103000000,1517,51900,40,15,1,1,3,INDEPENDENCIA OTE CENTRO .,339309,...,213,0,504,0,,0,0,2014-12-03 13:11:37.0,,
3,0600100104000000,1517,51900,40,15,1,1,3,INDEPENDENCIA OTE. No. 101 .,336433,...,283,0,566,0,,0,0,2014-12-03 13:11:37.0,,
4,0600100105000000,1517,51900,40,15,1,1,3,ALLENDE Y MINA CENTRO .,297891,...,170,0,185,0,,0,0,2014-12-03 13:11:37.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20625,0601399260000000,1517,51900,40,15,992,13,2,SAN ROMAN ARTURO S/N .,0,...,0,0,0,0,,0,0,2014-12-03 13:16:19.0,,
20626,0601399570000000,1517,51900,40,15,995,13,2,SAN ROMAN ARTURO S/N .,0,...,0,0,0,0,,0,0,2014-12-03 13:16:19.0,,
20627,0604031620000000,1517,51900,40,15,316,40,2,CON. SAN DIEGO ALCALA .,0,...,0,0,0,0,,0,0,2014-12-03 13:16:19.0,,
20628,0607101080000000,1517,51900,40,15,10,71,2,"DIONISIO IZQUIERDO COL.3 DE MAYO,IXTAPAN DE LA...",0,...,0,0,0,0,,0,0,2014-12-03 13:16:19.0,,


In [21]:
pred_pred['CLAVE_CATASTRAL']=pred_pred.CLAVECATASTRAL.str.slice(0,10)

In [22]:
pred_prop = pd.read_excel(r"D:\Secretaría\cruces_bases\Ixtapan de La Sal_PROPIETARIO.xlsx")
pred_prop

,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVECATASTRAL,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR
0,MUNICIPIO DE IXTAPAN DE LA SAL,12,2018,,51900,40,15,"PLAZA DE LOS MARTIRES, COL. CENTRO CP51900 ""JA...",0600100101000000,1,1517,,,,
1,MUNICIPIO DE IXTAPAN DE LA SAL,,,,51900,40,15,PLAZA DE LOS MARTIRES COL CENTRO CP 51900 PALA...,0600100102000000,1,1517,,,,
2,TENORIO DOMINGUEZ HERMELINDA,,,,51900,40,15,INDEPENDENCIA OTE CENTRO .,0600100103000000,1,1517,,,,
3,FIGUEROA DIAZ GUSTAVO,,,FIDG002120,51900,40,15,"CALLE INDEPENDENCIA NO.101COL. CENTRO, .",0600100104000000,1,1517,,,,
4,BENJAMIN IGNACIO ARAUJO DELGADO,,,ARDB630125,51900,40,15,ALLENDE Y MINA CENTRO .,0600100105000000,1,1517,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20625,ZONA UNO S DE RL DE CV,,,,51900,40,15,BOULEVARD SAN ROMAN ARTURO S/N .,0601399260000000,1,1517,,,,
20626,ZONA UNO S DE RL DE CV,,,,51900,40,15,BOULEVARD SAN ROMAN ARTURO S/N .,0601399570000000,1,1517,,,,
20627,FUENTES VARA MAXIMINO,,,,51900,40,15,CON. SAN DIEGO ALCALA .,0604031620000000,1,1517,,,,
20628,HERNANDEZ SALDAÃA MARIA,,,,51900,40,15,"20 DE NOVIEMBRE NO.24 BARRIO DE SANTACATARINA,...",0607101080000000,1,1517,,,,


In [23]:
pred_prop['CLAVE_CATASTRAL']=pred_prop.CLAVECATASTRAL.str.slice(0,10)
pred_prop

,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVECATASTRAL,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,CLAVE_CATASTRAL
0,MUNICIPIO DE IXTAPAN DE LA SAL,12,2018,,51900,40,15,"PLAZA DE LOS MARTIRES, COL. CENTRO CP51900 ""JA...",0600100101000000,1,1517,,,,,0600100101
1,MUNICIPIO DE IXTAPAN DE LA SAL,,,,51900,40,15,PLAZA DE LOS MARTIRES COL CENTRO CP 51900 PALA...,0600100102000000,1,1517,,,,,0600100102
2,TENORIO DOMINGUEZ HERMELINDA,,,,51900,40,15,INDEPENDENCIA OTE CENTRO .,0600100103000000,1,1517,,,,,0600100103
3,FIGUEROA DIAZ GUSTAVO,,,FIDG002120,51900,40,15,"CALLE INDEPENDENCIA NO.101COL. CENTRO, .",0600100104000000,1,1517,,,,,0600100104
4,BENJAMIN IGNACIO ARAUJO DELGADO,,,ARDB630125,51900,40,15,ALLENDE Y MINA CENTRO .,0600100105000000,1,1517,,,,,0600100105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20625,ZONA UNO S DE RL DE CV,,,,51900,40,15,BOULEVARD SAN ROMAN ARTURO S/N .,0601399260000000,1,1517,,,,,0601399260
20626,ZONA UNO S DE RL DE CV,,,,51900,40,15,BOULEVARD SAN ROMAN ARTURO S/N .,0601399570000000,1,1517,,,,,0601399570
20627,FUENTES VARA MAXIMINO,,,,51900,40,15,CON. SAN DIEGO ALCALA .,0604031620000000,1,1517,,,,,0604031620
20628,HERNANDEZ SALDAÃA MARIA,,,,51900,40,15,"20 DE NOVIEMBRE NO.24 BARRIO DE SANTACATARINA,...",0607101080000000,1,1517,,,,,0607101080


In [24]:
pred_deu = pd.read_excel(r"D:\Secretaría\cruces_bases\Ixtapan de La Sal_PREDIALADEUDO.xlsx")
pred_deu

,CLAVECATASTRAL,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS
0,0600100101000000,2019,1,12,3567030,Adeudo
1,0600100101000000,2020,1,12,3732269,Adeudo
2,0600100101000000,2021,1,12,3804383,Adeudo
3,0600100102000000,2009,1,12,1694038,Pago Municipal
4,0600100102000000,2010,1,12,1718011,Pago Municipal
...,...,...,...,...,...,...
146523,0600913503000000,2018,1,12,23310,Adeudo
146524,0600913503000000,2019,1,12,24512,Adeudo
146525,0600913503000000,2020,1,12,25900,Adeudo
146526,0600913503000000,2021,1,12,26418,Adeudo


In [25]:
def delete_empty(df):

    for i in df.columns:
        df.loc[(df[i] =="") |(df[i] =="0") | (df[i] ==0) | (df[i].isna()) | (df[i] ==" " ), i]=float('Nan')
    df.dropna(axis=1, how='all', inplace=True)
    
    return df 

In [26]:
delete_empty(pred_prop)

,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVECATASTRAL,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,CLAVE_CATASTRAL
0,MUNICIPIO DE IXTAPAN DE LA SAL,12,2018,NaN,51900.0,40.0,15.0,"PLAZA DE LOS MARTIRES, COL. CENTRO CP51900 ""JA...",0600100101000000,1.0,1517.0,NaN,NaN,NaN,NaN,0600100101
1,MUNICIPIO DE IXTAPAN DE LA SAL,NaN,NaN,NaN,51900.0,40.0,15.0,PLAZA DE LOS MARTIRES COL CENTRO CP 51900 PALA...,0600100102000000,1.0,1517.0,NaN,NaN,NaN,NaN,0600100102
2,TENORIO DOMINGUEZ HERMELINDA,NaN,NaN,NaN,51900.0,40.0,15.0,INDEPENDENCIA OTE CENTRO .,0600100103000000,1.0,1517.0,NaN,NaN,NaN,NaN,0600100103
3,FIGUEROA DIAZ GUSTAVO,NaN,NaN,FIDG002120,51900.0,40.0,15.0,"CALLE INDEPENDENCIA NO.101COL. CENTRO, .",0600100104000000,1.0,1517.0,NaN,NaN,NaN,NaN,0600100104
4,BENJAMIN IGNACIO ARAUJO DELGADO,NaN,NaN,ARDB630125,51900.0,40.0,15.0,ALLENDE Y MINA CENTRO .,0600100105000000,1.0,1517.0,NaN,NaN,NaN,NaN,0600100105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20625,ZONA UNO S DE RL DE CV,NaN,NaN,NaN,51900.0,40.0,15.0,BOULEVARD SAN ROMAN ARTURO S/N .,0601399260000000,1.0,1517.0,NaN,NaN,NaN,NaN,0601399260
20626,ZONA UNO S DE RL DE CV,NaN,NaN,NaN,51900.0,40.0,15.0,BOULEVARD SAN ROMAN ARTURO S/N .,0601399570000000,1.0,1517.0,NaN,NaN,NaN,NaN,0601399570
20627,FUENTES VARA MAXIMINO,NaN,NaN,NaN,51900.0,40.0,15.0,CON. SAN DIEGO ALCALA .,0604031620000000,1.0,1517.0,NaN,NaN,NaN,NaN,0604031620
20628,HERNANDEZ SALDAÃA MARIA,NaN,NaN,NaN,51900.0,40.0,15.0,"20 DE NOVIEMBRE NO.24 BARRIO DE SANTACATARINA,...",0607101080000000,1.0,1517.0,NaN,NaN,NaN,NaN,0607101080


In [27]:
delete_empty(pred_pred)

,CLAVECATASTRAL,CLAVEASENTAMIENTO,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CLAVEMANZANA,CLAVEZONA,CLAVESTATUS,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,SUPERFICIECONST,SUPERFICIECONSTCOMUN,CLAVEANTERIOR,FECHAALTA,CLAVE_CATASTRAL
0,0600100101000000,1517.0,51900.0,40.0,15.0,1.0,1.0,1.0,PLAZA LOS MARTIRES COL CENTRO S/N,3121950,NaN,610319,NaN,3147.0,NaN,133.0,NaN,NaN,2020-10-07 14:45:58.0,0600100101
1,0600100102000000,1517.0,51900.0,40.0,15.0,1.0,1.0,1.0,PLAZA DE LOS MARTIRES COL CENTRO CP 51900 PALA...,1086957,NaN,3332127,NaN,462.0,NaN,876.0,NaN,NaN,2014-12-03 13:11:37.0,0600100102
2,0600100103000000,1517.0,51900.0,40.0,15.0,1.0,1.0,3.0,INDEPENDENCIA OTE CENTRO .,339309,NaN,1256457,NaN,213.0,NaN,504.0,NaN,NaN,2014-12-03 13:11:37.0,0600100103
3,0600100104000000,1517.0,51900.0,40.0,15.0,1.0,1.0,3.0,INDEPENDENCIA OTE. No. 101 .,336433,NaN,1513303,NaN,283.0,NaN,566.0,NaN,NaN,2014-12-03 13:11:37.0,0600100104
4,0600100105000000,1517.0,51900.0,40.0,15.0,1.0,1.0,3.0,ALLENDE Y MINA CENTRO .,297891,NaN,311502,NaN,170.0,NaN,185.0,NaN,NaN,2014-12-03 13:11:37.0,0600100105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20625,0601399260000000,1517.0,51900.0,40.0,15.0,992.0,13.0,2.0,SAN ROMAN ARTURO S/N .,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-03 13:16:19.0,0601399260
20626,0601399570000000,1517.0,51900.0,40.0,15.0,995.0,13.0,2.0,SAN ROMAN ARTURO S/N .,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-03 13:16:19.0,0601399570
20627,0604031620000000,1517.0,51900.0,40.0,15.0,316.0,40.0,2.0,CON. SAN DIEGO ALCALA .,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-03 13:16:19.0,0604031620
20628,0607101080000000,1517.0,51900.0,40.0,15.0,10.0,71.0,2.0,"DIONISIO IZQUIERDO COL.3 DE MAYO,IXTAPAN DE LA...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-12-03 13:16:19.0,0607101080


In [28]:
pred_deu_2021 = pred_deu[pred_deu['EJERCICIO'] == 2021]

In [29]:
pred_deu_2021

,CLAVECATASTRAL,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS
2,0600100101000000,2021,1,12,3804383,Adeudo
15,0600100102000000,2021,1,12,4419084,Adeudo
28,0600100103000000,2021,1,12,2542477,Pago Municipal
35,0600100104000000,2021,1,12,2158407,Pago Municipal
42,0600100105000000,2021,1,12,815220,Pago Municipal
...,...,...,...,...,...,...
146504,0600913401000000,2021,1,12,63750,Adeudo
146511,0600913501000000,2021,1,12,202394,Pago Municipal
146518,0600913502000000,2021,1,12,5355,Adeudo
146526,0600913503000000,2021,1,12,26418,Adeudo


In [41]:
#pred_pred['CLAVE_CATASTRAL']= pred_pred.CLAVECATASTRAL.str.slice(0,10)
#pred_prop['CLAVE_CATASTRAL']= pred_pred.CLAVECATASTRAL.str.slice(0,10)
pred_deu_2021['CLAVE_CATASTRAL']= pred_deu_2021.CLAVECATASTRAL.str.slice(0,10)


,NOMBRE,APATERNO,AMATERNO,RFC,CLAVECP,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVECATASTRAL,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,CLAVE_CATASTRAL


In [31]:
pred_deu_2021

,CLAVECATASTRAL,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,CLAVE_CATASTRAL
2,0600100101000000,2021,1,12,3804383,Adeudo,0600100101
15,0600100102000000,2021,1,12,4419084,Adeudo,0600100102
28,0600100103000000,2021,1,12,2542477,Pago Municipal,0600100103
35,0600100104000000,2021,1,12,2158407,Pago Municipal,0600100104
42,0600100105000000,2021,1,12,815220,Pago Municipal,0600100105
...,...,...,...,...,...,...,...
146504,0600913401000000,2021,1,12,63750,Adeudo,0600913401
146511,0600913501000000,2021,1,12,202394,Pago Municipal,0600913501
146518,0600913502000000,2021,1,12,5355,Adeudo,0600913502
146526,0600913503000000,2021,1,12,26418,Adeudo,0600913503


In [44]:
delete_empty(Ixta_curt)

,CLAVE_CATASTRAL,No. CONSECUTIVO,DOMICILIO,NOMBRE DEL PROPIETARIO,RFC,ESTATUS DE LA CLAVE,VALOR CATASTRAL,SUPERFICIE DE TERRENO,USO DE ÁREA HOMOGÉNEA,PERIODO DE RECAUDACION CORRIENTE REZAGO,...,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,CLAVECATASTRAL,LATITUD,LONGITUD
0,0600100103,3.0,INDEPENDENCIA OTE CENTRO,TENORIO DOMINGUEZ HERMELINDA,NaN,2.0,213.0,718.0,8259.92,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
1,0600100104,4.0,"CALLE INDEPENDENCIA NO.101COL. CENTRO,",FIGUEROA DIAZ GUSTAVO,FIDG002120,2.0,283.0,577.0,6371.40,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
2,0600100105,5.0,NaN,BENJAMIN IGNACIO ARAUJO DELGADO,ARDB630125,2.0,170.0,178.0,1297.60,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
3,0600100106,6.0,J. MINA NO 5,ROSALINDA SOTELO DIAZ LEAL,NaN,2.0,96.0,177.0,2532.12,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
4,0600100107,7.0,ALLENDE,DIAZ MONTERO RUTH,DIDS001631,2.0,55.0,100.0,597.64,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13849,0240122320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15,024,0000,0240122320000000,194719536809952058052,NaN,POINT (-99.86827 19.78876),0240122320000000,19.788760,-99.868271
13850,0240122321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15,024,0000,0240122321000000,194720252909952063672,NaN,POINT (-99.86837 19.78889),0240122321000000,19.788893,-99.868374
13851,0240122328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15,024,0000,0240122328000000,194720252909952058206,NaN,POINT (-99.86834 19.78903),0240122328000000,19.789027,-99.868342
13852,0240122345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15,024,0000,0240122345000000,194720239609952072994,NaN,POINT (-99.86869 19.78896),0240122345000000,19.788957,-99.868694


Qué columnas están dentro de mi base x cruzar, que no están dentro de la 1ra base 

In [32]:
pred_pred.columns[pred_pred.columns.isin(pred_prop.columns)]
falta= pred_pred.columns[~pred_pred.columns.isin(pred_prop.columns)]
falta

Index(['CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA'],
      dtype='object')

Se hace el cruce con las columnas que no están dentro de la 1er base + la llave

In [33]:
predial_general = pd.merge(pred_pred[['CLAVECATASTRAL','CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA']],pred_prop, on="CLAVECATASTRAL", how="inner")
predial_general

,CLAVECATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,...,CLAVEMUNICIPIO,CLAVEENTIDAD,CALLE,CLAVESTATUS,CLAVEASENTAMIENTO,EMAIL,CURP,ASENTAMIENTO_NR,CLAVECP_NR,CLAVE_CATASTRAL
0,0600100101000000,1.0,1.0,PLAZA LOS MARTIRES COL CENTRO S/N,3121950,NaN,610319,NaN,3147.0,NaN,...,40.0,15.0,"PLAZA DE LOS MARTIRES, COL. CENTRO CP51900 ""JA...",1.0,1517.0,NaN,NaN,NaN,NaN,0600100101
1,0600100102000000,1.0,1.0,PLAZA DE LOS MARTIRES COL CENTRO CP 51900 PALA...,1086957,NaN,3332127,NaN,462.0,NaN,...,40.0,15.0,PLAZA DE LOS MARTIRES COL CENTRO CP 51900 PALA...,1.0,1517.0,NaN,NaN,NaN,NaN,0600100102
2,0600100103000000,1.0,1.0,INDEPENDENCIA OTE CENTRO .,339309,NaN,1256457,NaN,213.0,NaN,...,40.0,15.0,INDEPENDENCIA OTE CENTRO .,1.0,1517.0,NaN,NaN,NaN,NaN,0600100103
3,0600100104000000,1.0,1.0,INDEPENDENCIA OTE. No. 101 .,336433,NaN,1513303,NaN,283.0,NaN,...,40.0,15.0,"CALLE INDEPENDENCIA NO.101COL. CENTRO, .",1.0,1517.0,NaN,NaN,NaN,NaN,0600100104
4,0600100105000000,1.0,1.0,ALLENDE Y MINA CENTRO .,297891,NaN,311502,NaN,170.0,NaN,...,40.0,15.0,ALLENDE Y MINA CENTRO .,1.0,1517.0,NaN,NaN,NaN,NaN,0600100105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20625,0601399260000000,992.0,13.0,SAN ROMAN ARTURO S/N .,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,15.0,BOULEVARD SAN ROMAN ARTURO S/N .,1.0,1517.0,NaN,NaN,NaN,NaN,0601399260
20626,0601399570000000,995.0,13.0,SAN ROMAN ARTURO S/N .,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,15.0,BOULEVARD SAN ROMAN ARTURO S/N .,1.0,1517.0,NaN,NaN,NaN,NaN,0601399570
20627,0604031620000000,316.0,40.0,CON. SAN DIEGO ALCALA .,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,15.0,CON. SAN DIEGO ALCALA .,1.0,1517.0,NaN,NaN,NaN,NaN,0604031620
20628,0607101080000000,10.0,71.0,"DIONISIO IZQUIERDO COL.3 DE MAYO,IXTAPAN DE LA...",NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,15.0,"20 DE NOVIEMBRE NO.24 BARRIO DE SANTACATARINA,...",1.0,1517.0,NaN,NaN,NaN,NaN,0607101080


In [52]:
predial_general.columns[predial_general.columns.isin(pred_deu_2021.columns)]
falta_deu= predial_general.columns[~predial_general.columns.isin(pred_deu_2021.columns)]
falta_deu

Index(['CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC',
       'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS',
       'CLAVEASENTAMIENTO', 'EMAIL', 'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR'],
      dtype='object')

In [53]:
cruce_pred1=pd.merge(predial_general[['CLAVECATASTRAL','CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC',
       'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS',
       'CLAVEASENTAMIENTO', 'EMAIL', 'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR']],pred_deu_2021, on="CLAVECATASTRAL",  how='outer', indicator=True)
#cruce_pred1[cruce_pred1['_merge'] == 'left_only']
cruce_pred1

,CLAVECATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,...,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,CLAVE_CATASTRAL,_merge
0,0600100101000000,1.0,1.0,PLAZA LOS MARTIRES COL CENTRO S/N,3121950,NaN,610319,NaN,3147.0,NaN,...,NaN,NaN,NaN,2021.0,1.0,12.0,3804383.0,Adeudo,0600100101,both
1,0600100102000000,1.0,1.0,PLAZA DE LOS MARTIRES COL CENTRO CP 51900 PALA...,1086957,NaN,3332127,NaN,462.0,NaN,...,NaN,NaN,NaN,2021.0,1.0,12.0,4419084.0,Adeudo,0600100102,both
2,0600100103000000,1.0,1.0,INDEPENDENCIA OTE CENTRO .,339309,NaN,1256457,NaN,213.0,NaN,...,NaN,NaN,NaN,2021.0,1.0,12.0,2542477.0,Pago Municipal,0600100103,both
3,0600100104000000,1.0,1.0,INDEPENDENCIA OTE. No. 101 .,336433,NaN,1513303,NaN,283.0,NaN,...,NaN,NaN,NaN,2021.0,1.0,12.0,2158407.0,Pago Municipal,0600100104,both
4,0600100105000000,1.0,1.0,ALLENDE Y MINA CENTRO .,297891,NaN,311502,NaN,170.0,NaN,...,NaN,NaN,NaN,2021.0,1.0,12.0,815220.0,Pago Municipal,0600100105,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20625,0601399260000000,992.0,13.0,SAN ROMAN ARTURO S/N .,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
20626,0601399570000000,995.0,13.0,SAN ROMAN ARTURO S/N .,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
20627,0604031620000000,316.0,40.0,CON. SAN DIEGO ALCALA .,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
20628,0607101080000000,10.0,71.0,"DIONISIO IZQUIERDO COL.3 DE MAYO,IXTAPAN DE LA...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [54]:
cruce_pred1['CLAVE_CATASTRAL']=cruce_pred1.CLAVECATASTRAL.str.slice(0,10)
cruce_pred1

,CLAVECATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,SUPERFICIETERRCOMUN,...,CURP,ASENTAMIENTO_NR,CLAVECP_NR,EJERCICIO,CLAVEPERIODOINI,CLAVEPERIODOFIN,VALORCATASTRAL,STATUS,CLAVE_CATASTRAL,_merge
0,0600100101000000,1.0,1.0,PLAZA LOS MARTIRES COL CENTRO S/N,3121950,NaN,610319,NaN,3147.0,NaN,...,NaN,NaN,NaN,2021.0,1.0,12.0,3804383.0,Adeudo,0600100101,both
1,0600100102000000,1.0,1.0,PLAZA DE LOS MARTIRES COL CENTRO CP 51900 PALA...,1086957,NaN,3332127,NaN,462.0,NaN,...,NaN,NaN,NaN,2021.0,1.0,12.0,4419084.0,Adeudo,0600100102,both
2,0600100103000000,1.0,1.0,INDEPENDENCIA OTE CENTRO .,339309,NaN,1256457,NaN,213.0,NaN,...,NaN,NaN,NaN,2021.0,1.0,12.0,2542477.0,Pago Municipal,0600100103,both
3,0600100104000000,1.0,1.0,INDEPENDENCIA OTE. No. 101 .,336433,NaN,1513303,NaN,283.0,NaN,...,NaN,NaN,NaN,2021.0,1.0,12.0,2158407.0,Pago Municipal,0600100104,both
4,0600100105000000,1.0,1.0,ALLENDE Y MINA CENTRO .,297891,NaN,311502,NaN,170.0,NaN,...,NaN,NaN,NaN,2021.0,1.0,12.0,815220.0,Pago Municipal,0600100105,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20625,0601399260000000,992.0,13.0,SAN ROMAN ARTURO S/N .,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0601399260,left_only
20626,0601399570000000,995.0,13.0,SAN ROMAN ARTURO S/N .,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0601399570,left_only
20627,0604031620000000,316.0,40.0,CON. SAN DIEGO ALCALA .,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0604031620,left_only
20628,0607101080000000,10.0,71.0,"DIONISIO IZQUIERDO COL.3 DE MAYO,IXTAPAN DE LA...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0607101080,left_only


In [57]:
predial_completo = cruce_pred1[cruce_pred1['_merge'] =='both']
predial_sin_adeudos = cruce_pred1[cruce_pred1['_merge'] =='left_only']

In [ ]:
predial_completo

In [ ]:
predial_completo.to_excel(r"D:\Secretaría\cruces_bases\Predial_bases_cruce/predial_completo_ixtapan_2022.xlsx")
#predial_sin_adeudos.to_excel(r"D:\Secretaría\cruces_bases\Predial_bases_cruce/predial_no_adeudos_ixtapan_1.xlsx")

In [58]:
predial_completo=predial_completo.drop(['_merge'], axis=1)

In [59]:
predial_completo.columns[predial_completo.columns.isin(Ixta_curt.columns)]
falta_pred_completo= predial_completo.columns[~predial_completo.columns.isin(Ixta_curt.columns)]
falta_pred_completo

Index(['CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO', 'AMATERNO',
       'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS',
       'CLAVEASENTAMIENTO', 'EMAIL', 'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR',
       'EJERCICIO', 'CLAVEPERIODOINI', 'CLAVEPERIODOFIN', 'VALORCATASTRAL',
       'STATUS'],
      dtype='object')

In [68]:
cruce_CURT_predial = pd.merge(predial_completo[['CLAVE_CATASTRAL','CLAVECATASTRAL','CLAVEMANZANA', 'CLAVEZONA', 'DIRECCION', 'VALORTERRENO',
       'VALORTERRENOCOMUN', 'VALORCONSTPROP', 'VALORCONSTCOMUN', 'SUPERFICIE',
       'SUPERFICIETERRCOMUN', 'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN',
       'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO', 'AMATERNO',
       'CLAVECP', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS',
       'CLAVEASENTAMIENTO', 'EMAIL', 'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR',
       'EJERCICIO', 'CLAVEPERIODOINI', 'CLAVEPERIODOFIN', 'VALORCATASTRAL',
       'STATUS']], Ixta_curt, on='CLAVE_CATASTRAL', how = 'outer', indicator=True)
cruce_CURT_predial

,CLAVE_CATASTRAL,CLAVECATASTRAL_x,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,...,cve_mun,cve_loc,id_cat,curt,notas,geometry,CLAVECATASTRAL_y,LATITUD,LONGITUD,_merge
0,0600100101,0600100101000000,1.0,1.0,PLAZA LOS MARTIRES COL CENTRO S/N,3121950,NaN,610319,NaN,3147.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only
1,0600100102,0600100102000000,1.0,1.0,PLAZA DE LOS MARTIRES COL CENTRO CP 51900 PALA...,1086957,NaN,3332127,NaN,462.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only
2,0600100103,0600100103000000,1.0,1.0,INDEPENDENCIA OTE CENTRO .,339309,NaN,1256457,NaN,213.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,both
3,0600100104,0600100104000000,1.0,1.0,INDEPENDENCIA OTE. No. 101 .,336433,NaN,1513303,NaN,283.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,both
4,0600100105,0600100105000000,1.0,1.0,ALLENDE Y MINA CENTRO .,297891,NaN,311502,NaN,170.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70107,0240122320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,024,0000,0240122320000000,194719536809952058052,NaN,POINT (-99.86827 19.78876),0240122320000000,19.788760,-99.868271,right_only
70108,0240122321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,024,0000,0240122321000000,194720252909952063672,NaN,POINT (-99.86837 19.78889),0240122321000000,19.788893,-99.868374,right_only
70109,0240122328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,024,0000,0240122328000000,194720252909952058206,NaN,POINT (-99.86834 19.78903),0240122328000000,19.789027,-99.868342,right_only
70110,0240122345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,024,0000,0240122345000000,194720239609952072994,NaN,POINT (-99.86869 19.78896),0240122345000000,19.788957,-99.868694,right_only


In [63]:
###Ya no se usa, porque los que tenían NaN en x se mantenían, ya que se quedaba solo el de la x y no el de ya y

# cruce_CURT_predial['CLAVECATASTRAL']=cruce_CURT_predial.CLAVECATASTRAL_x.combine_first(cruce_CURT_predial.CLAVECATASTRAL_y)
# cruce_CURT_predial.drop(columns=['CLAVECATASTRAL_x','CLAVECATASTRAL_y'],inplace=True)

In [108]:
#Se reemplaza el valor de CLAVECATASTRAL_x por la de CLAVECATASTRAL_y en el caso de que sea Nan en x
cruce_CURT_predial.loc[cruce_CURT_predial.CLAVECATASTRAL_x.isna(),'CLAVECATASTRAL_x'] = 'CLAVECATASTRAL_y'

In [109]:
#Se verifica que ya no haya Nan's sobre CLAVECATASTRAL_x
cruce_CURT_predial[cruce_CURT_predial.CLAVECATASTRAL_x.isna()]

,CLAVE_CATASTRAL,CLAVECATASTRAL_x,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,...,cve_mun,cve_loc,id_cat,curt,notas,geometry,CLAVECATASTRAL_y,LATITUD,LONGITUD,_merge


In [110]:
cruce_CURT_predial = cruce_CURT_predial.rename(columns = {'_merge' : 'cruce_predial_2_CURT', 'CLAVECATASTRAL_x':'CLAVECATASTRAL'})
cruce_CURT_predial

,CLAVE_CATASTRAL,CLAVECATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,...,cve_mun,cve_loc,id_cat,curt,notas,geometry,CLAVECATASTRAL_y,LATITUD,LONGITUD,cruce_predial_2_CURT
0,0600100101,0600100101000000,1.0,1.0,PLAZA LOS MARTIRES COL CENTRO S/N,3121950,NaN,610319,NaN,3147.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only
1,0600100102,0600100102000000,1.0,1.0,PLAZA DE LOS MARTIRES COL CENTRO CP 51900 PALA...,1086957,NaN,3332127,NaN,462.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,left_only
2,0600100103,0600100103000000,1.0,1.0,INDEPENDENCIA OTE CENTRO .,339309,NaN,1256457,NaN,213.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,both
3,0600100104,0600100104000000,1.0,1.0,INDEPENDENCIA OTE. No. 101 .,336433,NaN,1513303,NaN,283.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,both
4,0600100105,0600100105000000,1.0,1.0,ALLENDE Y MINA CENTRO .,297891,NaN,311502,NaN,170.0,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70107,0240122320,0240122320000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,024,0000,0240122320000000,194719536809952058052,NaN,POINT (-99.86827 19.78876),0240122320000000,19.788760,-99.868271,right_only
70108,0240122321,0240122321000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,024,0000,0240122321000000,194720252909952063672,NaN,POINT (-99.86837 19.78889),0240122321000000,19.788893,-99.868374,right_only
70109,0240122328,0240122328000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,024,0000,0240122328000000,194720252909952058206,NaN,POINT (-99.86834 19.78903),0240122328000000,19.789027,-99.868342,right_only
70110,0240122345,0240122345000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,024,0000,0240122345000000,194720239609952072994,NaN,POINT (-99.86869 19.78896),0240122345000000,19.788957,-99.868694,right_only


In [112]:
cruce_CURT_predial.drop(['CLAVECATASTRAL_y'],axis=1,inplace=True )

In [115]:
cruce_CURT_predial[cruce_CURT_predial.LATITUD.isna()]

,CLAVE_CATASTRAL,CLAVECATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,...,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,LATITUD,LONGITUD,cruce_predial_2_CURT


Agregar LAT LON a los registros que no venían en la base de CURT

In [114]:
cruce_CURT_predial.loc[cruce_CURT_predial['LATITUD'].isna(), 'LATITUD'] = 18
cruce_CURT_predial.loc[cruce_CURT_predial['LONGITUD'].isna(), 'LONGITUD'] = -99

In [117]:
#cruce_CURT_predial.to_excel(r"D:\Secretaría\cruces_bases\Predial_bases_cruce/cruce_predial_curt_2022.xlsx")
cruce_CURT_predial.to_csv(r"D:\Secretaría\cruces_bases\Predial_bases_cruce/cruce_predial_curt.csv",encoding='utf-8-sig',index=False)


In [2]:
cruce_CURT_predial = pd.read_csv(r"D:\Secretaría\cruces_bases\Predial_bases_cruce/cruce_predial_curt.csv",encoding='utf-8-sig')
cruce_CURT_predial

,CLAVE_CATASTRAL,CLAVECATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,...,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,LATITUD,LONGITUD,cruce_predial_2_CURT
0,600100101,0600100101000000,1.0,1.0,PLAZA LOS MARTIRES COL CENTRO S/N,3121950.0,NaN,610319.0,NaN,3147.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.000000,-99.000000,left_only
1,600100102,0600100102000000,1.0,1.0,PLAZA DE LOS MARTIRES COL CENTRO CP 51900 PALA...,1086957.0,NaN,3332127.0,NaN,462.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.000000,-99.000000,left_only
2,600100103,0600100103000000,1.0,1.0,INDEPENDENCIA OTE CENTRO .,339309.0,NaN,1256457.0,NaN,213.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.000000,-99.000000,both
3,600100104,0600100104000000,1.0,1.0,INDEPENDENCIA OTE. No. 101 .,336433.0,NaN,1513303.0,NaN,283.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.000000,-99.000000,both
4,600100105,0600100105000000,1.0,1.0,ALLENDE Y MINA CENTRO .,297891.0,NaN,311502.0,NaN,170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.000000,-99.000000,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70107,240122320,0240122320000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.0,24.0,0.0,2.401223e+14,194719536809952058052,NaN,POINT (-99.86827118653642 19.788759847784984),19.788760,-99.868271,right_only
70108,240122321,0240122321000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.0,24.0,0.0,2.401223e+14,194720252909952063672,NaN,POINT (-99.86837408446873 19.788893082420362),19.788893,-99.868374,right_only
70109,240122328,0240122328000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.0,24.0,0.0,2.401223e+14,194720252909952058206,NaN,POINT (-99.86834235221059 19.78902712479704),19.789027,-99.868342,right_only
70110,240122345,0240122345000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.0,24.0,0.0,2.401223e+14,194720239609952072994,NaN,POINT (-99.8686936466193 19.78895672981456),19.788957,-99.868694,right_only


In [3]:
cruce_CURT_predial[cruce_CURT_predial.geometry.isna()]

,CLAVE_CATASTRAL,CLAVECATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,...,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,LATITUD,LONGITUD,cruce_predial_2_CURT
0,600100101,0600100101000000,1.0,1.0,PLAZA LOS MARTIRES COL CENTRO S/N,3121950.0,NaN,610319.0,NaN,3147.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,-99.0,left_only
1,600100102,0600100102000000,1.0,1.0,PLAZA DE LOS MARTIRES COL CENTRO CP 51900 PALA...,1086957.0,NaN,3332127.0,NaN,462.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,-99.0,left_only
2,600100103,0600100103000000,1.0,1.0,INDEPENDENCIA OTE CENTRO .,339309.0,NaN,1256457.0,NaN,213.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,-99.0,both
3,600100104,0600100104000000,1.0,1.0,INDEPENDENCIA OTE. No. 101 .,336433.0,NaN,1513303.0,NaN,283.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,-99.0,both
4,600100105,0600100105000000,1.0,1.0,ALLENDE Y MINA CENTRO .,297891.0,NaN,311502.0,NaN,170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,-99.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68183,600904104,CLAVECATASTRAL_y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,-99.0,right_only
68184,600904703,CLAVECATASTRAL_y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,-99.0,right_only
68185,600905005,CLAVECATASTRAL_y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,-99.0,right_only
68186,600905006,CLAVECATASTRAL_y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,-99.0,right_only


In [118]:
#cruce_CURT_predial['ESTIMADO']=cruce_CURT_predial.CLAVE_CATASTRAL.value_counts().reset_index()
base_temp=cruce_CURT_predial.groupby('CLAVE_CATASTRAL').count()[['CLAVECATASTRAL','curt']].reset_index()
base_temp

,CLAVE_CATASTRAL,CLAVECATASTRAL,curt
0,0240100601,1,1
1,0240100822,1,1
2,0240100827,1,1
3,0240100828,1,1
4,0240101114,1,1
...,...,...,...
18756,0600913401,1,0
18757,0600913501,1,0
18758,0600913502,1,0
18759,0600913503,1,0


In [119]:
base_temp=base_temp.rename(columns={'curt':'CURT'})
base_temp

,CLAVE_CATASTRAL,CLAVECATASTRAL,CURT
0,0240100601,1,1
1,0240100822,1,1
2,0240100827,1,1
3,0240100828,1,1
4,0240101114,1,1
...,...,...,...
18756,0600913401,1,0
18757,0600913501,1,0
18758,0600913502,1,0
18759,0600913503,1,0


In [122]:
base_temp.rename(columns={'CLAVECATASTRAL':'ESTIMADO'},inplace=True)

In [123]:
crucexcurt=cruce_CURT_predial.merge(base_temp[['ESTIMADO','CURT','CLAVE_CATASTRAL']],on='CLAVE_CATASTRAL')
crucexcurt

,CLAVE_CATASTRAL,CLAVECATASTRAL,CLAVEMANZANA,CLAVEZONA,DIRECCION,VALORTERRENO,VALORTERRENOCOMUN,VALORCONSTPROP,VALORCONSTCOMUN,SUPERFICIE,...,cve_loc,id_cat,curt,notas,geometry,LATITUD,LONGITUD,cruce_predial_2_CURT,ESTIMADO,CURT
0,0600100101,0600100101000000,1.0,1.0,PLAZA LOS MARTIRES COL CENTRO S/N,3121950,NaN,610319,NaN,3147.0,...,NaN,NaN,NaN,NaN,None,18.000000,-99.000000,left_only,1,0
1,0600100102,0600100102000000,1.0,1.0,PLAZA DE LOS MARTIRES COL CENTRO CP 51900 PALA...,1086957,NaN,3332127,NaN,462.0,...,NaN,NaN,NaN,NaN,None,18.000000,-99.000000,left_only,1,0
2,0600100103,0600100103000000,1.0,1.0,INDEPENDENCIA OTE CENTRO .,339309,NaN,1256457,NaN,213.0,...,NaN,NaN,NaN,NaN,None,18.000000,-99.000000,both,1,0
3,0600100104,0600100104000000,1.0,1.0,INDEPENDENCIA OTE. No. 101 .,336433,NaN,1513303,NaN,283.0,...,NaN,NaN,NaN,NaN,None,18.000000,-99.000000,both,1,0
4,0600100105,0600100105000000,1.0,1.0,ALLENDE Y MINA CENTRO .,297891,NaN,311502,NaN,170.0,...,NaN,NaN,NaN,NaN,None,18.000000,-99.000000,both,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70107,0240122320,0240122320000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0000,0240122320000000,194719536809952058052,NaN,POINT (-99.86827 19.78876),19.788760,-99.868271,right_only,1,1
70108,0240122321,0240122321000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0000,0240122321000000,194720252909952063672,NaN,POINT (-99.86837 19.78889),19.788893,-99.868374,right_only,1,1
70109,0240122328,0240122328000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0000,0240122328000000,194720252909952058206,NaN,POINT (-99.86834 19.78903),19.789027,-99.868342,right_only,1,1
70110,0240122345,0240122345000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0000,0240122345000000,194720239609952072994,NaN,POINT (-99.86869 19.78896),19.788957,-99.868694,right_only,1,1


In [124]:
#crucexcurt.to_excel(r"D:\Secretaría\cruces_bases\Predial_bases_cruce\crucexcurt.xlsx")
crucexcurt.to_csv(r"D:\Secretaría\cruces_bases\Predial_bases_cruce\crucexcurt_Ixtapan.csv",encoding='utf-8-sig',index=False)

In [ ]:
# sys.path.append(r'C:\Users\mfpen\OneDrive\Documentos\Repositorios\geoloc2\puntos_sobre_poligono')
# from simulacion_interpolacion import *
# sys.path.append(r'C:\Users\mfpen\OneDrive\Documentos\Repositorios\geoloc2\preparacion_datos\src')
# from preparacion_inter_puntos import *

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import sys
from dataprep.clean import clean_lat_long

#from src.preparacion_inter_puntos import prep
sys.path.append(r'D:\Repositorios\geoloc2\puntos_sobre_poligono')
from simulacion_interpolacion import *
sys.path.append(r'D:\Repositorios\geoloc2\preparacion_datos\src')
from preparacion_inter_puntos import prep
'''Interpolación de puntos'''

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\mfpen\AppData\Local\Temp\ipykernel_7876\1572859696.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedo

'Interpolación de puntos'

In [2]:
#PATH_BASE = Base final del cruce de predial x curt (con estimado)
PATH_BASE = r'D:\Secretaría\cruces_bases\Atlacomulco\crucexcurt_atlacomulco.csv'
PATH_SHP  = r"D:\Secretaría\Bases_shp_INEGI\CURT\024 Atlacomulco\ATLACOMULCO_PREDIOS.shp"
df_final_catastro = post_points_catastro(PATH_BASE, PATH_SHP, task_chunks)
df_final_catastro.to_csv(r'D:\Secretaría\cruces_bases\Predial_bases_cruce\puntos/Ixtapan_puntos.csv', encoding='utf-8-sig')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mfpen\AppData\Local\Temp\ipykernel_7876\3951019991.py", line 4, in <module>
    df_final_catastro = post_points_catastro(PATH_BASE, PATH_SHP, task_chunks)
  File "D:\Repositorios\geoloc2\puntos_sobre_poligono\simulacion_interpolacion.py", line 425, in post_points_catastro
    test_igecem = prep.data_prep_catastro(path_base, path_shp)
  File "D:\Repositorios\geoloc2\preparacion_datos\src\preparacion_inter_puntos.py", line 231, in data_prep_catastro
    BPCE = pd.read_csv(path_base, encoding="utf-8",
  File "C:\Users\mfpen\AppData\Roaming\Python\Python39\site-packages\pandas\util\_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\mfpen\AppData\Roaming\Python\Python39\site-packages\pandas\util\_decorators.py", line 331, in wrapper


In [49]:
cambio = gpd.read_file(r"D:\Secretaría\cruces_bases\Atlacomulco\atlacomulco_salidas\atlacomulco_file.shp")
cambio.columns

Index(['Unnamed_ 0', 'CLAVE_CATA', 'CLAVEMANZA', 'CLAVEZONA', 'DIRECCION',
       'VALORTERRE', 'VALORTER_1', 'VALORCONST', 'VALORCON_1', 'SUPERFICIE',
       'SUPERFIC_1', 'SUPERFIC_2', 'SUPERFIC_3', 'CLAVEANTER', 'FECHAALTA',
       'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC', 'CLAVECP', 'CLAVEMUNIC',
       'CLAVEENTID', 'CALLE', 'CLAVESTATU', 'CLAVEASENT', 'EMAIL', 'CURP',
       'ASENTAMIEN', 'CLAVECP_NR', 'EJERCICIO', 'CLAVEPERIO', 'CLAVEPER_1',
       'VALORCATAS', 'STATUS', 'fol_rec', 'nombre_1', 'rfc_1', 'ubicacion',
       'domicilio_', 'sup_terren', 'sup_constr', 'valor_cata', 'periodo',
       'periodo_an', 'fec_pago', 'importe', 'recargos', 'multas', 'descuento',
       'fol_fisc_u', 'facturado', 'cve_ent', 'cve_mun', 'cve_loc', 'id_cat',
       'CURT', 'notas', 'LATITUD', 'LONGITUD', '_merge', 'CLAVECATAS',
       'ESTIMADO', 'CONTEO', 'geometry'],
      dtype='object')

In [39]:
dict_buenas=['Unnamed: 0', 'CLAVE_CATASTRAL', 'CLAVEMANZANA', 'CLAVEZONA',
       'DIRECCION', 'VALORTERRENO', 'VALORTERRENOCOMUN', 'VALORCONSTPROP',
       'VALORCONSTCOMUN', 'SUPERFICIE', 'SUPERFICIETERRCOMUN',
       'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN', 'CLAVEANTERIOR', 'FECHAALTA',
       'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC', 'CLAVECP', 'CLAVEMUNICIPIO',
       'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS', 'CLAVEASENTAMIENTO', 'EMAIL',
       'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR', 'EJERCICIO', 'CLAVEPERIODOINI',
       'CLAVEPERIODOFIN', 'VALORCATASTRAL', 'STATUS', 'fol_rec', 'nombre',
       'rfc', 'ubicacion', 'domicilio_fiscal', 'sup_terreno', 'sup_construc',
       'valor_catastral', 'periodo', 'periodo_anual', 'fec_pago', 'importe',
       'recargos', 'multas', 'descuento', 'fol_fisc_uuid', 'facturado',
       'cve_ent', 'cve_mun', 'cve_loc', 'id_cat', 'CURT', 'notas', 
       'LATITUD', 'LONGITUD', '_merge', 'CLAVECATASTRAL', 'ESTIMADO',
       'CONTEO','geometry']

dict_malas = ['Unnamed_ 0', 'CLAVE_CATA', 'CLAVEMANZA', 'CLAVEZONA', 'DIRECCION',
       'VALORTERRE', 'VALORTER_1', 'VALORCONST', 'VALORCON_1', 'SUPERFICIE',
       'SUPERFIC_1', 'SUPERFIC_2', 'SUPERFIC_3', 'CLAVEANTER', 'FECHAALTA',
       'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC', 'CLAVECP', 'CLAVEMUNIC',
       'CLAVEENTID', 'CALLE', 'CLAVESTATU', 'CLAVEASENT', 'EMAIL', 'CURP',
       'ASENTAMIEN', 'CLAVECP_NR', 'EJERCICIO', 'CLAVEPERIO', 'CLAVEPER_1',
       'VALORCATAS', 'STATUS', 'fol_rec', 'nombre_1', 'rfc_1', 'ubicacion',
       'domicilio_', 'sup_terren', 'sup_constr', 'valor_cata', 'periodo',
       'periodo_an', 'fec_pago', 'importe', 'recargos', 'multas', 'descuento',
       'fol_fisc_u', 'facturado', 'cve_ent', 'cve_mun', 'cve_loc', 'id_cat',
       'CURT', 'notas', 'LATITUD', 'LONGITUD', '_merge', 'CLAVECATAS',
       'ESTIMADO', 'CONTEO', 'geometry']

In [59]:
dos=cambio

In [60]:
def replace_columns(df):
    
    for i in range(len(column_malos)):
        for j in range(len(column_buenos)):
            n= column_malos[i]
            if n in column_buenos[j]:

                df.rename(columns={n:column_buenos[j]},inplace=True)
    return df

Index(['Unnamed_ 0', 'CLAVE_CATASTRAL', 'CLAVEMANZANA', 'CLAVEZONA',
       'DIRECCION', 'VALORTERRENO', 'VALORTER_1', 'VALORCONSTPROP',
       'VALORCON_1', 'SUPERFICIETERRCOMUN', 'SUPERFIC_1', 'SUPERFIC_2',
       'SUPERFIC_3', 'CLAVEANTERIOR', 'FECHAALTA', 'NOMBRE', 'APATERNO',
       'AMATERNO', 'RFC', 'CLAVECP_NR', 'CLAVEMUNICIPIO', 'CLAVEENTIDAD',
       'CALLE', 'CLAVESTATUS', 'CLAVEASENTAMIENTO', 'EMAIL', 'CURP',
       'CLAVEASENTAMIENTO', 'CLAVECP_NR', 'EJERCICIO', 'CLAVEPERIODOINI',
       'CLAVEPER_1', 'VALORCATASTRAL', 'CLAVESTATUS', 'fol_rec', 'nombre_1',
       'rfc_1', 'ubicacion', 'domicilio_fiscal', 'sup_terreno', 'sup_construc',
       'valor_catastral', 'periodo_anual', 'periodo_anual', 'fec_pago',
       'importe', 'recargos', 'multas', 'descuento', 'fol_fisc_uuid',
       'facturado', 'cve_ent', 'cve_mun', 'cve_loc', 'id_cat', 'CURT', 'notas',
       'LATITUD', 'LONGITUD', '_merge', 'CLAVECATASTRAL', 'ESTIMADO', 'CONTEO',
       'geometry', 's'],
      dtype='obje

In [57]:
d=['Unnamed_ 0', 'CLAVE_CATA', 'CLAVEMANZA', 'CLAVEZONA', 'DIRECCION',
       'VALORTERRE', 'VALORTER_1', 'VALORCONST', 'VALORCON_1', 'SUPERFICIE',
       'SUPERFIC_1', 'SUPERFIC_2', 'SUPERFIC_3', 'CLAVEANTER', 'FECHAALTA',
       'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC', 'CLAVECP', 'CLAVEMUNIC',
       'CLAVEENTID', 'CALLE', 'CLAVESTATU', 'CLAVEASENT', 'EMAIL', 'CURP',
       'ASENTAMIEN', 'CLAVECP_NR', 'EJERCICIO', 'CLAVEPERIO', 'CLAVEPER_1',
       'VALORCATAS', 'STATUS', 'fol_rec', 'nombre_1', 'rfc_1', 'ubicacion',
       'domicilio_', 'sup_terren', 'sup_constr', 'valor_cata', 'periodo',
       'periodo_an', 'fec_pago', 'importe', 'recargos', 'multas', 'descuento',
       'fol_fisc_u', 'facturado', 'cve_ent', 'cve_mun', 'cve_loc', 'id_cat',
       'CURT', 'notas', 'LATITUD', 'LONGITUD', '_merge', 'CLAVECATAS',
       'ESTIMADO', 'CONTEO', 'geometry']
d1=['Unnamed_ 0', 'CLAVE_CATASTRAL', 'CLAVEMANZANA', 'CLAVEZONA',
       'DIRECCION', 'VALORTERRENO', 'VALORTERRENOCOMUN', 'VALORCONSTPROP',
       'VALORCONSTCOMUN', 'SUPERFICIE', 'SUPERFICIETERRCOMUN',
       'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN', 'CLAVEANTERIOR', 'FECHAALTA',
       'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC', 'CLAVECP', 'CLAVEMUNICIPIO',
       'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS', 'CLAVEASENTAMIENTO', 'EMAIL',
       'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR', 'EJERCICIO', 'CLAVEPERIODOINI',
       'CLAVEPERIODOFIN', 'VALORCATASTRAL', 'STATUS', 'fol_rec', 'nombre_1',
       'rfc', 'ubicacion', 'domicilio_fiscal', 'sup_terreno', 'sup_construc',
       'valor_catastral', 'periodo', 'periodo_anual', 'fec_pago', 'importe',
       'recargos', 'multas', 'descuento', 'fol_fisc_uuid', 'facturado',
       'cve_ent', 'cve_mun', 'cve_loc', 'id_cat', 'CURT', 'notas', 
       'LATITUD', 'LONGITUD', '_merge', 'CLAVECATASTRAL', 'ESTIMADO',
       'CONTEO','geometry']
d2=[str(x)[:10] for x in d1]
#d3=pd.DataFrame()
for i in d2:
    uno.loc[d.index(i),"s"]=d1[d2.index(i)]

uno.sort_index(inplace=True)
uno

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mfpen\AppData\Local\Temp\ipykernel_19912\598609303.py", line 30, in <module>
    uno.loc[d.index(i),"s"]=d1[d2.index(i)]
ValueError: 'rfc' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\ultratb.py", line 1288, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\ultratb.py", line 1177, in structured_traceback
    return VerboseTB.structured_traceback(
  File "c

In [ ]:
'Unnamed_ 0':'Unnamed: 0', 'CLAVE_CATA': 'CLAVE_CATASTRAL', 'CLAVEMANZA':'CLAVEMANZANA', 'CLAVEZONA':'CLAVEZONA', 'DIRECCION':,
       'VALORTERRE':, 'VALORTER_1':, 'VALORCONST':, 'VALORCON_1':, 'SUPERFICIE':,
       'SUPERFIC_1':, 'SUPERFIC_2':, 'SUPERFIC_3':, 'CLAVEANTER':, 'FECHAALTA':,
       'NOMBRE', 'APATERNO', 'AMATERNO':, 'RFC':, 'CLAVECP':, 'CLAVEMUNIC':,
       'CLAVEENTID', 'CALLE', 'CLAVESTATU', 'CLAVEASENT', 'EMAIL', 'CURP',
       'ASENTAMIEN', 'CLAVECP_NR', 'EJERCICIO', 'CLAVEPERIO', 'CLAVEPER_1',
       'VALORCATAS', 'STATUS', 'fol_rec', 'nombre_1', 'rfc_1', 'ubicacion',
       'domicilio_', 'sup_terren', 'sup_constr', 'valor_cata', 'periodo',
       'periodo_an', 'fec_pago', 'importe', 'recargos', 'multas', 'descuento',
       'fol_fisc_u', 'facturado', 'cve_ent', 'cve_mun', 'cve_loc', 'id_cat',
       'CURT', 'notas', 'LATITUD', 'LONGITUD', '_merge', 'CLAVECATAS',
       'ESTIMADO', 'CONTEO', 'geometry'

In [ ]:
dicc_columnas_atla = [, , , ,
       'DIRECCION', 'VALORTERRENO', 'VALORTERRENOCOMUN', 'VALORCONSTPROP',
       'VALORCONSTCOMUN', 'SUPERFICIE', 'SUPERFICIETERRCOMUN',
       'SUPERFICIECONST', 'SUPERFICIECONSTCOMUN', 'CLAVEANTERIOR', 'FECHAALTA',
       'NOMBRE', 'APATERNO', 'AMATERNO', 'RFC', 'CLAVECP', 'CLAVEMUNICIPIO',
       'CLAVEENTIDAD', 'CALLE', 'CLAVESTATUS', 'CLAVEASENTAMIENTO', 'EMAIL',
       'CURP', 'ASENTAMIENTO_NR', 'CLAVECP_NR', 'EJERCICIO', 'CLAVEPERIODOINI',
       'CLAVEPERIODOFIN', 'VALORCATASTRAL', 'STATUS', 'fol_rec', 'nombre',
       'rfc', 'ubicacion', 'domicilio_fiscal', 'sup_terreno', 'sup_construc',
       'valor_catastral', 'periodo', 'periodo_anual', 'fec_pago', 'importe',
       'recargos', 'multas', 'descuento', 'fol_fisc_uuid', 'facturado',
       'cve_ent', 'cve_mun', 'cve_loc', 'id_cat', 'CURT', 'notas', 'geometry',
       'LATITUD', 'LONGITUD', '_merge', 'CLAVECATASTRAL', 'ESTIMADO',
       'CONTEO']

In [5]:
import pandas as pd
import geopandas as gpd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from shapely.geometry import MultiPoint
from tqdm import tqdm
import shapely
import concurrent.futures
import os
from simulacion_interpolacion import prep
# import sys
# sys.path.append(r'C:\Users\mfpen\OneDrive\Documentos\Repositorios\geoloc2\puntos_sobre_poligono')
# from simulacion_interpolacion import task_chunks
# sys.path.append(r'C:\Users\mfpen\OneDrive\Documentos\Repositorios\geoloc2\preparacion_datos\src')
# from preparacion_inter_puntos import prep
# '''Interpolación de puntos'''

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mfpen\AppData\Local\Temp\ipykernel_17976\571928244.py", line 11, in <module>
    from simulacion_interpolacion import prep
ModuleNotFoundError: No module named 'simulacion_interpolacion'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\ultratb.py", line 1288, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\IPython\core\ultratb.py", line 1177, in structured_traceback
    return VerboseT

In [ ]:
PATH_BASE = r"D:\Secretaría\cruces_bases\Atlacomulco\crucexcurt_atlacomulco.xlsx"
PATH_SHP  = r"D:\Secretaría\cruces_bases\curt_ixtapan_completa.shp"
#test_igecem = pd.read_csv(r'D:\Secretaría\cruces_bases\Predial_bases_cruce\puntos\test_igecem_ch.csv', encoding='utf-8-sig')
test_igecem = prep.data_prep_catastro(PATH_BASE, PATH_SHP)
print('test igecem es: ',test_igecem)
# else:
#     m_igecem = gpd.read_file(path_shp) ##Lee desde shp

# #     m_igecem.crs= 4326 #m_igecem.to_crs(4326)
# try:
#     m_igecem = m_igecem.loc[~m_igecem['manz'].astype(str).str.endswith('000')]
# except: 
   #test_igecem = m_igecem
test_igecem_chunks =  np.array_split(test_igecem, os.cpu_count()-1) ##Aqui se especifica si se requiere un loc y los chunks

df_concat = pd.DataFrame()
funcion = task_chunks
with concurrent.futures.ThreadPoolExecutor() as executor:
    for df_chunk in tqdm(zip(executor.map(funcion, test_igecem_chunks[1])),total = len(test_igecem_chunks[1])):
        df_concat = pd.concat([df_chunk[0],df_concat], axis=0)


In [15]:
test_igecem.to_csv(r'D:\Secretaría\cruces_bases\Predial_bases_cruce\puntos/test_igecem_ch_atla.csv', encoding='utf-8-sig')